In [1]:
# sanity check
import dgl
import torch
import dgl.function as fn

dev = torch.device(f"cuda:{torch.cuda.device_count()-1}"
                   if torch.cuda.is_available() else "cpu")
print(dev)

# dgl cuda
src_ids = torch.tensor([2,3,4])
dst_ids = torch.tensor([1,2,3])
g_test = dgl.graph((src_ids, dst_ids))
g_test = g_test.to(dev)

c:\ProgramData\Anaconda3\envs\diffuser\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\Anaconda3\envs\diffuser\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


cpu


In [3]:
import numpy as np
from sklearn.metrics import f1_score
from transformers import TrainingArguments, DataCollatorWithPadding
from transformers import RobertaTokenizer
from transformers.utils import logging
from datasets import load_dataset,load_metric,load_from_disk
from models.diffuser_utils import DiffuserConfig
from graphtrainer import graphTrainer

logging.set_verbosity_debug()
logger = logging.get_logger()

def preprocess_function(examples):
    return tokenizer(examples['text'], padding = 'max_length', truncation=True, max_length = 1024)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    f1_score = metric_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": acc, "f1_score": f1_score }

metric_acc = load_metric('./metrics/accuracy')
metric_f1 = load_metric('./metrics/f1')

imdb = load_dataset("imdb")
# tokenizer = RobertaTokenizer.from_pretrained("./roberta-tokenizer", max_length = 1024)
tokenizer = RobertaTokenizer(tokenizer_file = "./roberta-tokenizer/tokenizer.json",
                             vocab_file     = "./roberta-tokenizer/vocab.json",
                             merges_file    = "./roberta-tokenizer/merges.txt",
                             max_length     = 1024)

tokenized_imdb = imdb.map(preprocess_function, batched=True)
tokenized_imdb = tokenized_imdb.map(remove_columns=["text"])
# tokenized_imdb = load_from_disk("/home")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Reusing dataset imdb (C:\Users\Kevin Qu\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|██████████| 3/3 [00:01<00:00,  2.96it/s]
Parameter 'function'=<function preprocess_function at 0x000001FD65649700> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 50000/50000 [01:07<00:00, 741.21ex/s]


In [4]:
#config = DiffuserConfig.from_json_file("./models/config.json")
config = DiffuserConfig.from_json_file("./models/config_trial.json")
config.num_labels = 2
with_frac = False

In [5]:
config

DiffuserConfig {
  "attention_mode": "diffuser",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    64,
    64,
    64
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "Diffuser",
  "num_attention_heads": 6,
  "num_glob": 64,
  "num_hidden_layers": 3,
  "num_rand": 64,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [6]:
type(config)

models.diffuser_utils.DiffuserConfig

In [7]:
attn_setup = {"with_frac":True, "gamma":0.5}

In [8]:
# sanity check 2
from models.diffuser_att import DiffuserAttention
attention_block = DiffuserAttention(config, **attn_setup)
print(attention_block.with_frac)

True


In [ ]:
from models.diffuser_app import DiffuserForSequenceClassification
model =  DiffuserForSequenceClassification(config, **attn_setup)
model = model.to(dev)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\diffuser\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Kevin Qu\AppData\Local\Temp\ipykernel_14200\1128065411.py", line 3, in <module>
    model =  DiffuserForSequenceClassification(config, **attn_setup)
  File "y:\Diffuser\models\diffuser_app.py", line 73, in __init__
    self.classifier = DiffuserClassificationHead(config)
  File "y:\Diffuser\models\diffuser.py", line 54, in __init__
    self.pooler = DiffuserPooler(config) if add_pooling_layer else None
  File "y:\Diffuser\models\diffuser.py", line 207, in __init__
  File "y:\Diffuser\models\diffuser.py", line 207, in <listcomp>
  File "y:\Diffuser\models\diffuser.py", line 305, in __init__
    self.output = DiffuserOutput(config)
  File "y:\Diffuser\models\diffuser_att.py", line 28, in __init__
    else:
AttributeError: 'float' object has no attribute 'is_numer

In [ ]:
model.diffuser.encoder.layer[1]

DiffuserLayer(
  (attention): DiffuserAttention(
    (self): DiffuserSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
    )
    (output): DiffuserSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): DiffuserIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): DiffuserOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [17]:
training_args = TrainingArguments(
    output_dir = "./save_imdb",
    learning_rate = 3e-5,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 1,
    weight_decay = 0.01,
    evaluation_strategy = "steps",
    eval_steps = 2,
    logging_steps = 500,
    save_steps = 500,
    seed = 42,
    warmup_steps = 200,
    gradient_accumulation_steps = 8,
    prediction_loss_only=True
)

if dev.type != "cpu":
    steps_per_train_epoch       = int(len(tokenized_imdb['train'])/(training_args.per_device_train_batch_size*torch.cuda.device_count()*training_args.gradient_accumulation_steps ))
else:
    steps_per_train_epoch       = int(len(tokenized_imdb['train'])/(training_args.per_device_train_batch_size*torch.get_num_threads()*training_args.gradient_accumulation_steps ))
training_args.eval_steps    = int(steps_per_train_epoch)
training_args.logging_steps = int(steps_per_train_epoch/5)
training_args.save_steps    = int(steps_per_train_epoch)

trainer = graphTrainer(
    model = model,
    config = config,
    args = training_args,
    train_dataset = tokenized_imdb["train"],
    eval_dataset = tokenized_imdb["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

#trainer.train()

TypeError: __init__() takes 2 positional arguments but 3 were given

In [ ]:
max_len = 4096
attention_window = (
    config.attention_window
    if isinstance(config.attention_window, int)
    else max(config.attention_window)
)
n_blocks = max_len//(attention_window//2)-1
adj = np.zeros([max_len, max_len])
# add local window att (overlap)
for i in range(n_blocks):
    start = i*attention_window//2
    end = start+attention_window
    if end > max_len:
        end = max_len
    adj[start:end, start:end] = 1

# add random att
np.random.seed(0)
num_random = max_len*config.num_rand

NameError: name 'config' is not defined

In [ ]:
idx = np.random.choice(range(max_len*max_len), num_random ,replace=False)
idx_x = idx %  max_len
idx_y = idx // max_len
adj[idx_x,idx_y] = 1

# add global att
num_global = config.num_glob
idx = np.random.choice(range(attention_window,max_len), num_global ,replace=False)
adj[idx,:] = 1
adj[:,idx] = 1

possible_seq_len = np.arange(attention_window, max_len+attention_window, attention_window)
src_dst = {k: np.nonzero(adj[:k, :k]) for k in possible_seq_len}

In [4]:
B, N, H, D = 5, 64, 3, 2
batch_size, seq_len = B, N
g_list = []
for i in range(B):
    src,dst = src_dst[seq_len]
    g = dgl.graph((src, dst))
    g_list.append(g)
batched_g = dgl.batch(g_list)

NameError: name 'src_dst' is not defined

In [ ]:
qvecs = torch.ones([B, N, H, D])
kvecs = torch.ones([B, N, H, D])
vvecs = torch.ones([B, N, H, D])
batched_g.ndata['q'] = qvecs.reshape(-1, H, D)
batched_g.ndata['k'] = kvecs.reshape(-1, H, D)
batched_g.ndata['v'] = vvecs.reshape(-1, H, D)

In [ ]:
batched_g.ndata['q'].shape

In [ ]:
from models.utils import *
batched_g.apply_edges(fn.u_dot_v('k', 'q', 'score'))   #score: [E,H,1]
batched_g.apply_edges(mask_attention_score)   #kq
e = batched_g.edata.pop('score')
batched_g.edata['score'] = edge_softmax(batched_g, e)
batched_g.edata['score']= nn.functional.dropout(batched_g.edata['score'],
                                                p=config.attention_probs_dropout_prob,
                                                training=False)

batched_g.ndata["h"] = batched_g.ndata["v"]